In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('document.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]


{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [7]:
q = 'the course has already started, can I still enroll?'

In [8]:
index.fit(documents)

In [9]:
from openai import OpenAI

In [10]:
client = OpenAI()

In [11]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content


"It's great that you're interested in the course! Whether you can still enroll depends on the specific institution or platform offering the course. Here are a few steps you can take:\n\n1. **Check the Course Information**: Look for details about enrollment deadlines on the course webpage or in any materials you have.\n\n2. **Contact the Instructor or Administration**: Reach out to the course instructor or the administration office to ask if late enrollment is possible. Explain your situation and express your enthusiasm for the course.\n\n3. **Review Policies**: Some institutions allow late enrollment with certain conditions, such as a grace period or additional fees. Make sure to understand any implications of enrolling late.\n\n4. **Online Platforms**: If the course is through an online platform like Coursera, Udacity, or edX, check their specific policies. Many online courses are self-paced or have flexible enrollment windows.\n\n5. **Catch Up**: If you are allowed to enroll late, be

In [12]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [15]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [16]:
rag(query)

'To run Kafka components such as a producer, consumer, KStreams, etc., in the terminal, follow these steps:\n\n1. Navigate to your project directory.\n2. Execute the following command:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\nReplace `<jar_name>` with the actual name of the JAR file generated for your project.\n\nThis command will run the Java application that you have specified, which could be a producer, consumer, or any other Kafka component you have implemented.'

In [17]:
rag('the course has already started, can I still enroll?')

'Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homework assignments. Make sure to keep an eye on the final project deadlines, as it’s important not to leave everything until the last minute.'

In [18]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
from elasticsearch import Elasticsearch


In [20]:
es_client = Elasticsearch('http://localhost:9200') 